# MRI Residual Convolutional Neural Network
This is the resnet model for the AD pre-diagnosis with artificial intelligence. Uses MRI data, which is 3D. Uses pretrained [medicalnet](https://github.com/Tencent/MedicalNet).

In [ ]:
import os
import glob

import torch
import torch.nn as nn

import pandas as pd
from skimage import io, transform
from sklearn import preprocessing
from torchvision import transforms, utils
import adabound
import numpy as np

import nibabel as nib
import random

import torchio

In [ ]:
import os
os.chdir("MedicalNet/")
import model as mn
os.chdir("../")

In [ ]:
dir(mn)

In [ ]:
# Use the GPU if there is one, otherwise CPU
DEVICE = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')

CSV_DIR = "./scores.csv"
DATA_DIR = "./mri_data/"

In [ ]:
df = pd.read_csv(CSV_DIR)
norm_df = df[["mmse", "cdr", "ageAtEntry"]]

std_scale = preprocessing.StandardScaler().fit(norm_df)

## Dataset Management
Handle CSV diagnosis/signs and get an iterator of brain scans

In [ ]:
def get_scores(ID, date):
    scores = []
    for index, row in df[df["Subject"].str.contains(ID)].iterrows():
        cur_date = int(row["ADRC_ADRCCLINICALDATA ID"].split("_")[-1][1:])
        if cur_date > date:
            if cur_date > date:
                if pd.isna(row["mmse"]): row["mmse"] = 30
                if pd.isna(row["cdr"]): row["cdr"] = 0
                data = {
                    'mmse':  [row["mmse"]],
                    'cdr':  [row["cdr"]],
                    'ageAtEntry': [row["ageAtEntry"]+cur_date/365]
                }

                curr_df = std_scale.transform(pd.DataFrame(data, columns=["mmse", "cdr", "ageAtEntry"]))

                scores.append((cur_date-date, curr_df[0][0], curr_df[0][1], curr_df[0][2]))
    
    return scores

In [ ]:
get_scores('OAS30001', 129)  # testing

In [ ]:
def list_brains():
    for file in glob.glob(f"{DATA_DIR}OAS3[0-9][0-9][0-9][0-9]_MR_d[0-9][0-9][0-9][0-9]/anat[0-9]/sub-OAS3[0-9][0-9][0-9][0-9]_ses-d[0-9][0-9][0-9][0-9]_run-[0-9][0-9]_T1w.nii.gz"):
        yield (file, file.split("/")[2].split("_")[0], int(file.split("/")[2].split("_")[2][1:]))

def get_brains():
    for brain in list_brains():
        for score in get_scores(brain[1], brain[2]):
            yield (brain[0], brain[1], brain[2]) + score

## Define Dataset
Create an iterable dataset with brain data inheriting from `torch.utils.data.Dataset`. Dataset len is `3594`.

In [ ]:
class BrainsDataset(torch.utils.data.Dataset):
    def __init__(self, transform=None):
        self.transform = transform
        self.brains = []
        brain_files = []
        for brain_name in get_brains():
            if get_scores(*brain_name[1:3]) != []:
                self.brains.append(brain_name)
                brain_files.append(torchio.Subject(
                    torchio.Image("MRI", brain_name[0], torchio.INTENSITY),
                ))
        self.images_dataset = torchio.ImagesDataset(brain_files)

    def __len__(self):
        return len(self.brains)
    
    def __getitem__(self, index):
        return self.brains[index][3], self.brains[index][4], self.brains[index][5], self.brains[index][6], self.transform(self.images_dataset[index])

## Create Data Preprocessing and Cropping and Data Augmentation
Crop images to (128, 128, 63, 24).

In [ ]:
class Rescaledef:
    """Rescale the image in a sample to a given size."""


    def __init__(self, output_size):
        self.output_size = output_size

    def __call__(self, brain):
        img = transform.resize(brain, self.output_size)

        return img


class ToTensor:
    """Convert ndarrays in sample to Tensors."""

    def __call__(self, sample):
        return sample["MRI"]["data"][:, :, :, :]

class RandomPermute:
    def __call__(self, sample):
        dim = [1, 2, 3]
        random.shuffle(dim)
        return sample.permute(0, *dim)

from torchio.transforms import (
    ZNormalization,
    RandomNoise,
    RandomFlip,
    RandomAffine,
)

## Define Neural Network
Create a sparse cnn module inheriting from `torch.nn.Module`.

In [ ]:
class BidirectionalLSTM(nn.Module):

    def __init__(self, nIn, nHidden, nOut):
        super(BidirectionalLSTM, self).__init__()

        self.rnn = nn.LSTM(nIn, nHidden, bidirectional=True)
        self.embedding = nn.Linear(nHidden * 2, nOut)

    def forward(self, input):
        recurrent, _ = self.rnn(input)
        T, b, h = recurrent.size()
        t_rec = recurrent.view(T * b, h)

        output = self.embedding(t_rec)  # [T * b, nOut]
        output = output.view(T, b, -1)

        return output
    
class Net(nn.Module):
    def __init__(self, resnet):
        nn.Module.__init__(self)
        self.resnet = resnet.to(DEVICE)
        
        self.linear = nn.Sequential(
            nn.Linear(64002, 512),
            nn.BatchNorm1d(512),
            torch.nn.ELU(),
            nn.Linear(512, 512),
            nn.BatchNorm1d(512),
            torch.nn.ELU(),
            nn.Linear(512, 128),
            nn.BatchNorm1d(128),
            torch.nn.ELU(),
            nn.Linear(128, 16),
            nn.BatchNorm1d(16),
            torch.nn.ELU(),
            nn.Linear(16, 2)
        ).to(DEVICE)
        
        self.flatten = nn.Flatten(start_dim=1)
        self.avgpool = nn.AvgPool3d((4, 4, 4), stride=(3, 3, 3))
        self.maxpool = nn.AvgPool3d((3, 3, 3), stride=(2, 2, 2))

    def forward(self, brain, days_ahead, age):
        
        c_out = self.resnet(self.maxpool(brain))
        
        c_out = self.flatten(self.avgpool(c_out))
        
        c_out = torch.cat([torch.stack([days_ahead, age]).permute(1, 0), c_out], 1).to(DEVICE)
        r_out = torch.cuda.FloatTensor(self.linear(c_out)).to(DEVICE)
        return r_out


In [ ]:
def train(model, optimizer, criterion, criterion_test, train_loader, test_loader, writer):
    step_num = 0
    for epoch in range(NUM_EPOCHS):
        print(f"Epoch {epoch+1}/{NUM_EPOCHS}")
        print('-' * 10)
        running_loss = 0
        train_iter = iter(train_loader)
        
        for i, data_brains in enumerate(train_loader):
            step_num += 1
            scan = data_brains[4].type(torch.cuda.FloatTensor).to(DEVICE)
            days_ahead = data_brains[0].type(torch.cuda.FloatTensor).to(DEVICE)
            
            age = data_brains[3].type(torch.cuda.FloatTensor).to(DEVICE)
            real_values = torch.stack([data_brains[1], data_brains[2]]).permute(1, 0).to(DEVICE)

            outputs = model(scan, days_ahead, age)

            loss = criterion(outputs, real_values)

            optimizer.zero_grad()
            loss.backward()
            optimizer.step()
            torch.cuda.empty_cache()

            running_loss += loss.item()
            if i % 5 == 4:
                print(f"[{epoch + 1} {i + 1}] loss: {running_loss/5}")
                writer.add_scalar("Training Loss", running_loss/5, step_num)
                running_loss = 0


                with torch.no_grad():
                    
                    try:
                        test_data = next(train_iter)
                    except StopIteration:
                        train_iter = iter(train_loader)
                        test_data = next(train_iter)

                    _scan = test_data[4].type(torch.cuda.FloatTensor).to(DEVICE)
                    _days_ahead = test_data[0].type(torch.cuda.FloatTensor).to(DEVICE)
                    _age = test_data[3].type(torch.cuda.FloatTensor).to(DEVICE)
                    _real_values = torch.stack([test_data[1], test_data[2]]).permute(1, 0).to(DEVICE)
                    _outputs = model(_scan, _days_ahead, _age)
                    writer.add_scalar("Test Loss", criterion_test(_outputs, _real_values), step_num)

                torch.cuda.empty_cache()

        print("Saving model")
        torch.save(model.state_dict(), f"model_pretrained_cnn/new_model_{epoch}_cnn.pt")

In [ ]:
class Params():
    def __init__(self):
        self.model = "resnet"
        self.model_depth = 34
        self.input_W = 128
        self.input_H = 128
        self.input_D = 128
        self.resnet_shortcut = "A"
        self.no_cuda = False
        self.gpu_id = [0]
        self.n_seg_classes = 128
        self.phase = "train"
        self.pretrain_path = os.getcwd()+"/MedicalNet/pretrain/resnet_34_23dataset.pth"
        self.new_layer_names = ['upsample1', 'cmp_layer3', 'upsample2', 'cmp_layer2', 'upsample3', 'cmp_layer1', 'upsample4', 'cmp_conv1', 'conv_seg']

## Hyperparameters

In [ ]:
NUM_EPOCHS = 50
BATCH_SIZE = 3

## Get Data

In [ ]:
from torchio.transforms import (
    RandomFlip,
    RandomNoise,
    RandomMotion,
    RandomBiasField,
    Rescale,
    ToCanonical,
    CenterCropOrPad,
)

from torchvision.transforms import Compose

training_transform = Compose([
    Rescale((0, 1)),  # so that there are no negative values for RandomMotion
    RandomMotion(),
    RandomBiasField(),
    RandomNoise(std_range=(0, 0.005)),
    ToCanonical(),
    CenterCropOrPad((256, 256, 256)),
    RandomFlip(axes=(0,)),
    ToTensor(),
    RandomPermute()
])

dataset = BrainsDataset(training_transform)

NUM_INSTANCES = len(dataset)
TEST_RATIO = 0.2
TEST_SIZE = int(NUM_INSTANCES * TEST_RATIO)
TRAIN_SIZE = NUM_INSTANCES - TEST_SIZE

In [ ]:
train_data, test_data = torch.utils.data.random_split(dataset, (TRAIN_SIZE, TEST_SIZE))
train_loader = torch.utils.data.DataLoader(train_data, batch_size=BATCH_SIZE, shuffle = True, drop_last=True)
test_loader = torch.utils.data.DataLoader(test_data, batch_size=BATCH_SIZE, shuffle = False, drop_last=True)

In [ ]:
from torch.utils.tensorboard import SummaryWriter
model, _ = mn.generate_model(Params())

model = Net(model)

optimizer = torch.optim.AdamW(model.parameters())  
criterion = torch.nn.MSELoss()
criterion_test = torch.nn.MSELoss()
writer = SummaryWriter()

loss = train(model, optimizer, criterion, criterion_test, train_loader, test_loader, writer)

In [ ]:
print(sum(p.numel() for p in model.parameters() if p.requires_grad))

In [ ]:
for i, data_brains in enumerate(train_loader):
    scan = data_brains[4].type(torch.cuda.FloatTensor).to(DEVICE)
    days_ahead = data_brains[0].type(torch.cuda.FloatTensor).to(DEVICE)
    age = data_brains[3].type(torch.cuda.FloatTensor).to(DEVICE)

    writer.add_graph(model, (scan, days_ahead, age))
    break
writer.close()

In [ ]:
torch.save(model, f"model_cnn_normal_final_new.pt")

In [ ]:
from tensorboard.backend.event_processing.event_accumulator import EventAccumulator
event_acc = EventAccumulator("./runs/Mar02_08-24-04_linux-desktop/events.out.tfevents.1583155445.linux-desktop.23221.0")
event_acc.Reload()
# Show all tags in the log file

previous_step = 0
writer = SummaryWriter()
for i, loss in enumerate(event_acc.Scalars("Training_Loss")):
    writer.add_scalar("Training_Loss", loss.value, global_step=i, walltime=loss.wall_time,)

for i, loss in enumerate(event_acc.Scalars("Test_Loss")):
    writer.add_scalar("Test_Loss", loss.value, global_step=i, walltime=loss.wall_time,)

In [ ]:
torch.cuda.empty_cache()

## Test model

In [ ]:
with torch.no_grad():
    for i, data_brains in enumerate(train_loader):
        scan = data_brains[4].type(torch.cuda.FloatTensor).to(DEVICE)
        days_ahead = data_brains[0].type(torch.cuda.FloatTensor).to(DEVICE)
        age = data_brains[3].type(torch.cuda.FloatTensor).to(DEVICE)
        real_values = torch.stack([data_brains[1], data_brains[2]]).permute(1, 0).to(DEVICE)

        model.eval()
        outputs = model(scan[None, 0], days_ahead[None, 0], age[None, 0])
        print(f"Outputs: {outputs}")
        print(f"Real Values: {real_values}")

        data_pred = {
            'mmse':  [outputs[0][0]],
            'cdr':  [outputs[0][1]]
        }

        data_pred = std_scale.inverse_transform(pd.DataFrame(data_pred, columns=["mmse", "cdr", "ageAtEntry"]))

        data_real = {
            'mmse':  [real_values[0][0]],
            'cdr':  [real_values[0][1]]
        }

        data_real = std_scale.inverse_transform(pd.DataFrame(data_real, columns=["mmse", "cdr", "ageAtEntry"]))
    
        print(data_pred)
        print(data_real)

In [ ]:
model.train()
list(model.parameters())

# Get Dimensions of Data
Here we get the dimensions of the data to be cropped

Size: 
```
{
    (176, 256, 256): 3348, 
    (256, 256, 128): 1737, 
    (176, 240, 256): 56, 
    (175, 256, 256): 8
}
```

5149 Scans

In [ ]:
dimensions = {}

for brain in get_brains():
    shape = nib.load(brain[0]).get_fdata().shape
    if shape in dimensions:
        dimensions[shape] += 1
    else:
        dimensions[shape] = 1

print(dimensions)

nums = 0
for num in dimensions.values():
    nums+=num
nums

In [ ]:
smallest, largest = 0, 0

for brain in list_brains():
    data = nib.load(brain[0]).get_fdata()
    
    if np.min(data) < smallest:
        smallest = np.min(data)
    if int(np.max(data)) > largest:
        largest = np.max(data)

(smallest, largest)
# should be (-4401596.0, 4831864.0)

In [ ]:
for brain in get_brains():
    shape = nib.load(brain[0]).get_fdata().shape
    if len(shape) == 3:
        print(brain)

In [ ]:
bad_data = ['../data/sub-OAS30065/ses-d0553/pet/sub-OAS30065_ses-d0553_acq-PIB_pet.nii.gz',
'../data/sub-OAS30229/ses-d0101/pet/sub-OAS30229_ses-d0101_acq-PIB_pet.nii.gz',
'../data/sub-OAS30253/ses-d3948/pet/sub-OAS30253_ses-d3948_acq-PIB_pet.nii.gz',
'../data/sub-OAS30332/ses-d0091/pet/sub-OAS30332_ses-d0091_acq-PIB_pet.nii.gz',
'../data/sub-OAS30472/ses-d1278/pet/sub-OAS30472_ses-d1278_acq-PIB_pet.nii.gz',
'../data/sub-OAS30498/ses-d0120/pet/sub-OAS30498_ses-d0120_acq-PIB_pet.nii.gz',
'../data/sub-OAS30588/ses-d1639/pet/sub-OAS30588_ses-d1639_acq-PIB_pet.nii.gz',
'../data/sub-OAS30896/ses-d1601/pet/sub-OAS30896_ses-d1601_acq-PIB_pet.nii.gz'
]